In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

In [3]:
DEVICE = 'ibmq_mumbai'
DISTANCE = 3
ROUNDS = 3
_RESETS = False
LOGICAL = 0

In [4]:
from soft_info import find_longest_path_in_hex

backend = provider.get_backend(DEVICE)
longest_path, _, _ = find_longest_path_in_hex(backend)
print(longest_path)

17:45:41 Warning: Not a valid heavy hex map, found 6 cornernodes instead of 2. Looking for longest path starting with the last corner node.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/coupling_map.py, LINE: 135
[26, 25, 22, 19, 16, 14, 11, 8, 5, 3, 2, 1, 4, 7, 10, 12, 15, 18, 21, 23, 24]


In [8]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit import transpile
from tqdm import tqdm

circuits = []
for i in tqdm(range(len(longest_path)-(2*DISTANCE-1))):
    bounded_path = longest_path[i : i+2*DISTANCE-1]
    layout = bounded_path[1::2] + bounded_path[::2] 
    code = RepetitionCodeCircuit(DISTANCE, ROUNDS, xbasis=False, resets=_RESETS)
    qc = code.circuit[str(LOGICAL)]
    circuits.append(transpile(qc, backend, initial_layout=layout, optimization_level=1))

100%|██████████| 16/16 [00:00<00:00, 113.30it/s]


In [9]:
# get the rep_delay
rep_delay=backend.configuration().rep_delay_range[1]
print(rep_delay)

0.0005


In [13]:
from Scratch import metadata_helper

SHOTS = 1e6/(DISTANCE*ROUNDS)

metadata = metadata_helper(descr="Mean positions", code="RepetitionCodeCircuit", distance=DISTANCE,
                           rounds=ROUNDS, resets=_RESETS, logical=LOGICAL)
backend.run(metadata, circuits, shots=SHOTS/len(circuits), meas_level=1, meas_return='single', 
            job_tags=["RepCode Mean Positions"], rep_delay=rep_delay) 


<IBMCircuitJob('cqpyzcd2b6ng008y69pg')>